In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import time
import requests_cache 
import time
from IPython.core.display import clear_output

In [5]:
# keeping data from API calls into cache
requests_cache.install_cache()

**Who's in the ISS right now ?**

In [2]:
astronaut_api = requests.get("http://api.open-notify.org/astros.json")

astronaut_api.status_code

200

In [3]:
iss_person = astronaut_api.json()
iss_person

{'message': 'success',
 'number': 5,
 'people': [{'craft': 'ISS', 'name': 'Chris Cassidy'},
  {'craft': 'ISS', 'name': 'Anatoly Ivanishin'},
  {'craft': 'ISS', 'name': 'Ivan Vagner'},
  {'craft': 'ISS', 'name': 'Doug Hurley'},
  {'craft': 'ISS', 'name': 'Bob Behnken'}]}

In [4]:
print('Currently there are ', iss_person['number'], ' people in space')

Currently there are  5  people in space


In [7]:
# Create a loop to get the ISS info over 10 minutes every 5 seconds

ISS_loc = []
count = 0
while count <2160:
    print('The rocket is on the launch pad - looking for ISS location')
    print(f'ISS API call - request# {count}/2160')
    #API call
    
    # clear the output of each print to make things neater
    clear_output(wait = True)
    
    response = requests.get('http://api.open-notify.org/iss-now.json')
    status = response.status_code
    
    #print status code
    if status != 200:
        print(f'API call crashes - code is {status}')
    else:
        resp = response.json()
        #append each API response to ISS_loc list
        ISS_loc.append(resp)
        
        #10 sec pause btw each API call - allows ISS to move slightly
        time.sleep(5)
        
        count += 1

The rocket is on the launch pad - looking for ISS location
ISS API call - request# 2159/2160


In [10]:
ISS_loc[:5]

[{'timestamp': 1591262419,
  'message': 'success',
  'iss_position': {'latitude': '41.7587', 'longitude': '63.4514'}},
 {'timestamp': 1591262419,
  'message': 'success',
  'iss_position': {'latitude': '41.7587', 'longitude': '63.4514'}},
 {'timestamp': 1591262419,
  'message': 'success',
  'iss_position': {'latitude': '41.7587', 'longitude': '63.4514'}},
 {'timestamp': 1591262419,
  'message': 'success',
  'iss_position': {'latitude': '41.7587', 'longitude': '63.4514'}},
 {'timestamp': 1591262419,
  'message': 'success',
  'iss_position': {'latitude': '41.7587', 'longitude': '63.4514'}}]

In [11]:
ISS_loc[5]

{'timestamp': 1591262419,
 'message': 'success',
 'iss_position': {'latitude': '41.7587', 'longitude': '63.4514'}}

In [13]:
#initializing empty dict to populate
ISS_dict = {'latitude':None, 'longitude':None, 'timestamp':None}

# Initialize empty lists to populate with values from the API
lat_list = []
long_list = []
time_list = []

for i in range(len(ISS_loc)):
    lat_list.append(ISS_loc[i]['iss_position']['latitude'])
    long_list.append(ISS_loc[i]['iss_position']['longitude'])
    time_list.append(ISS_loc[i]['timestamp'])

#populate ISS dict
ISS_dict['latitude'] = lat_list
ISS_dict['longitude'] = long_list
ISS_dict['timestamp'] = time_list

In [20]:
ISS_df = pd.DataFrame(ISS_dict)

In [21]:
ISS_df

,latitude,longitude,timestamp
0,41.7587,63.4514,1591262419
1,41.7587,63.4514,1591262419
2,41.7587,63.4514,1591262419
3,41.7587,63.4514,1591262419
4,41.7587,63.4514,1591262419
...,...,...,...
2155,41.7587,63.4514,1591262419
2156,41.7587,63.4514,1591262419
2157,41.7587,63.4514,1591262419
2158,41.7587,63.4514,1591262419


In [22]:
# Change values to integers from strings
ISS_df['latitude'] = ISS_df['latitude'].astype(float)
ISS_df['longitude'] = ISS_df['longitude'].astype(float)
ISS_df['timestamp'] = ISS_df['timestamp'].astype(float)